<a href="https://colab.research.google.com/github/AashiDutt/AI-and-ML-for-Coders/blob/main/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains code for :
1. Fashion MNIST TensorFlow Datasets

In [1]:
pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
(training_images, training_labels),(test_images, test_labels) = tfds.as_numpy(tfds.load('fashion_mnist', 
                                                                                        split= ['train','test'],
                                                                                        batch_size= -1,
                                                                                        as_supervised = True))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteAXXUST/fashion_mnist-train.tfrecord


...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteAXXUST/fashion_mnist-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [4]:
training_images = training_images/255.0
test_images = test_images/255.0


In [5]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (28,28,1)),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                  ])
model.compile(optimizer= 'adam',
              loss= 'sparse_categorical_crossentropy',
              metrics= ['accuracy'])

In [6]:
model.fit(training_images, training_labels, epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5312 - accuracy: 0.8122
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3989 - accuracy: 0.8543
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3667 - accuracy: 0.8661
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3444 - accuracy: 0.8723
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3303 - accuracy: 0.8782


In [7]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.3603 - accuracy: 0.8668


[0.360341340303421, 0.8668000102043152]

In [8]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

[7.3822135e-05 1.0860820e-06 3.1190047e-02 3.0384520e-06 8.8346797e-01
 2.5366245e-09 8.5223749e-02 1.2289427e-09 4.0378927e-05 7.5976542e-10]
4
